In [71]:
import numba
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import convolve, generate_binary_structure
from matplotlib.animation import FuncAnimation
import matplotlib
matplotlib.use('QT5Agg')
%matplotlib qt

In [72]:
class metropolis:
    def __init__(self, N, J=1, B=0):
        self.J = J
        self.N = N
        self.init_random = np.random.random((N, N))
        self.grid_n = np.zeros((N, N))
        self.grid_n[self.init_random >= .5] = 1
        self.grid_n[self.init_random < .5] = -1
    
    def get_energy(self, grid):
        kernel = generate_binary_structure(2, 1)
        kernel[1][1] = False
        result = - self.J * grid * convolve(grid, kernel, mode='wrap')
        return result.sum()
    
    def grid(self):
        return self.grid_n
    
    @staticmethod
    @numba.njit("Tuple((f8[:], f8[:], f8[:,:]))(f8[:,:], i8, f8, f8, i8)", nopython=True, nogil=True)
    def algo(arr_spin, times, beta, energy, N):
        arr_spin = arr_spin.copy()
        total_spin = np.zeros(times - 1)
        total_energy = np.zeros(times - 1)
        #spin_states = np.zeros((times - 1, N, N))
        # x = np.random.randint(0, N, size=(times - 1))
        # y = np.random.randint(0, N, size=(times - 1))
        for t in range(0, times - 1):
            x = np.random.randint(0,N)
            y = np.random.randint(0,N)
            spin_t = arr_spin[x, y]
            spin_prime = -spin_t
            E_t = 0
            E_prime = 0
            # if x>0:
            #     E_t += - spin_t * arr_spin[x-1,y]
            #     E_prime += - spin_prime * arr_spin[x-1,y]
            # if x<N-1:
            #     E_t += - spin_t * arr_spin[x+1,y]
            #     E_prime += - spin_prime * arr_spin[x+1,y]
            # if y>0:
            #     E_t += - spin_t * arr_spin[x,y-1]
            #     E_prime += - spin_prime * arr_spin[x,y-1]
            # if y<N-1:
            #     E_t += - spin_t * arr_spin[x,y+1]
            #     E_prime += - spin_prime * arr_spin[x,y+1]
            neighbours = [(x - 1) % N, (x + 1) % N, (y - 1) % N, (y + 1) % N]
            for nx in [neighbours[0], neighbours[1]]:
                E_t += - spin_t * arr_spin[nx, y]
                E_prime += - spin_prime * arr_spin[nx, y]
            for ny in [neighbours[2], neighbours[3]]:
                E_t += - spin_t * arr_spin[x, ny]
                E_prime += - spin_prime * arr_spin[x, ny]
            dE = E_prime - E_t
            if (dE > 0) & (np.random.random() < np.exp(-beta * dE)):
                arr_spin[x, y] = spin_prime
                energy += dE
            elif dE <= 0:
                arr_spin[x, y] = spin_prime
                energy += dE
            total_spin[t] = arr_spin.sum()
            total_energy[t] = energy
            #spin_states[t] = arr_spin
            
        return total_spin, total_energy, arr_spin
        
        
    def print_energy(self):
        print(self.get_energy(self.grid_n))
    
    def plot(self):
        plt.imshow(self.grid_n, cmap='binary')

/Users/danielmiksch/miniconda3/envs/juplab/lib/python3.10/site-packages/numba/core/decorators.py:282: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [73]:
test = metropolis(N=50)

In [74]:
#test.plot()
test.print_energy()

168.0


In [75]:
spins = test.grid()
energy = test.get_energy(spins)

In [76]:
kb = 1.380649e-23
T = 1e21
beta = 1 / (kb * T)

In [77]:
spins, energies, equilibrium = test.algo(spins, 2500000, 0.45, energy, 50)

In [81]:
fig, axes = plt.subplots(1, 2, figsize=(12,4))

plt.rcParams["font.family"] = "times"
plt.rcParams["text.usetex"] = True

ax = axes[0]
ax.plot(spins/50**2)
ax.set_xlabel('Algorithm Time Steps')
ax.set_ylabel(r'Magnetization $\langle M\rangle$')
ax.grid()
ax = axes[1]
ax.plot(energies)
ax.set_xlabel('Algorithm Time Steps')
ax.set_ylabel(r'Energy $E$')
ax.grid()
fig.tight_layout()
fig.suptitle(r'Evolution of Average Spin and Energy', y=1.07, size=18)
plt.show()

In [79]:
fig, axes = plt.subplots()

plt.rcParams["font.family"] = "times"
plt.rcParams["text.usetex"] = True

plt.imshow(equilibrium, cmap='binary')

In [80]:
# fig, ax = plt.subplots()
# 
# def update(frame):
#     ax.clear()
#     ax.imshow(equilibrium[frame], cmap='binary')  # oder eine andere passende Colormap
#     ax.set_title(f"Zeitschritt {frame * 1000}")
# 
# ani = FuncAnimation(fig, update, 999999 // 1000, interval=1)
# plt.show()